In [2]:
import traceback
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import interact
import pandas as pd
import psycopg2 as pg
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy.sql.expression import text

## Criando conexão com o Banco Postgres

In [3]:
host = "localhost"
port = 5432
database = "postgres"
username = "postgres"
password = "2505"

cnx = f"postgresql://{username}:{password}@{host}:{port}/{database}"
eng = sqlalchemy.create_engine(cnx)
conn = eng.connect()

In [4]:
host = 'localhost'
dbname = 'postgres'
user = 'postgres'
password = '2505'

con = pg.connect(host=host, dbname=dbname, user=user, password=password)

In [5]:
# Testando conexão
try:
    result = conn.execute(text("SELECT d.nome, COUNT(f.id_func) AS total_funcionarios "
                               "FROM Departamento AS d "
                               "INNER JOIN Funcionario AS f ON d.id_dpto = f.id_dpto "
                               "GROUP BY d.nome"))

    for row in result:
        print(f"Departamento: {row.nome}  Total de Funcionários: {row.total_funcionarios}")

finally:
    conn.close()
    eng.dispose()

Departamento: Marketing Digital  Total de Funcionários: 1
Departamento: Logística  Total de Funcionários: 1
Departamento: Atendimento ao Cliente  Total de Funcionários: 1
Departamento: Tecnologia  Total de Funcionários: 2
Departamento: Vendas  Total de Funcionários: 1
Departamento: Administrativo  Total de Funcionários: 1
Departamento: Produção  Total de Funcionários: 1
Departamento: Qualidade  Total de Funcionários: 1
Departamento: Financeiro  Total de Funcionários: 1
Departamento: Recursos Humanos  Total de Funcionários: 1


## DEPARTAMENTO

In [37]:
# Widgets para a tabela Departamento
nome_dpto = widgets.Text(
    value='',
    description='Nome Departamento:',
    disabled=False
)

# Botão com descrição 
button_ins_dpto = widgets.Button(description="Inserir Departamento")

# Ação que será executada ao clicar no botão de inserir departamento
def on_button_insert_dpto(b):
    try:
        # Inserir departamento
        cursor = con.cursor()
        cursor.execute("INSERT INTO Departamento (nome) VALUES (%s)", (nome_dpto.value,))
        con.commit()

        with output_dpto:
            output_dpto.clear_output()
            print("Departamento inserido com sucesso!")

    except Exception as e:
        con.rollback()

        with output_dpto:
            output_dpto.clear_output()
            print("Não foi possível realizar a inserção do departamento. Verifique as restrições e tente novamente.")
            print("Erro:", str(e))
            traceback.print_exc()

    finally:
        cursor.close()

# Widget para listar departamentos
button_list_dpto = widgets.Button(description="Listar Departamentos")

# Ação que será executada ao clicar no botão de listar departamentos
def on_button_list_dpto(b):
    try:
        # Listar departamentos
        cursor = con.cursor()
        cursor.execute("SELECT id_dpto, nome FROM Departamento")
        result = cursor.fetchall()

        with output_dpto:
            output_dpto.clear_output()
            print("Departamentos:")
            for row in result:
                print("ID:", row[0])
                print("Nome:", row[1])
                print("")

    except Exception as e:
        with output_dpto:
            output_dpto.clear_output()
            print("Erro ao listar departamentos:", str(e))
            traceback.print_exc()

    finally:
        cursor.close()

# Associar ação com o botão de listar departamentos
button_list_dpto.on_click(on_button_list_dpto)

# Widget para deletar departamento
button_del_dpto = widgets.Button(description="Deletar Departamento")

# Ação que será executada ao clicar no botão de deletar departamento
def on_button_del_dpto(b):
    try:
        # Widgets para inserir o nome do departamento a ser deletado
        nome_dpto_del = widgets.Text(
            value='',
            description='Nome Departamento:',
            disabled=False
        )

        confirm_del_dpto = widgets.Button(description="Confirmar Deleção")

        def on_button_confirm_del_dpto(b):
            try:
                # Deletar departamento
                cursor = con.cursor()
                cursor.execute("DELETE FROM Departamento WHERE nome = %s", (nome_dpto_del.value,))
                con.commit()

                with output_dpto:
                    output_dpto.clear_output()
                    print("Departamento deletado com sucesso!")

            except Exception as e:
                con.rollback()

                with output_dpto:
                    output_dpto.clear_output()
                    print("Não foi possível deletar o departamento. Verifique o nome e tente novamente.")
                    print("Erro:", str(e))
                    traceback.print_exc()

            finally:
                cursor.close()

        # Associar ação com o botão de confirmar deleção
        confirm_del_dpto.on_click(on_button_confirm_del_dpto)

        # Exibir widgets para inserir nome do departamento e botão de confirmação
        with output_dpto:
            output_dpto.clear_output()
            display(nome_dpto_del, confirm_del_dpto)

    except Exception as e:
        with output_dpto:
            output_dpto.clear_output()
            print("Erro ao deletar departamento:", str(e))
            traceback.print_exc()

# Widget para atualizar departamento
button_update_dpto = widgets.Button(description="Atualizar Departamento")

# Ação que será executada ao clicar no botão de atualizar departamento
def on_button_update_dpto(b):
    try:
        # Widgets para inserir o nome do departamento a ser atualizado
        nome_dpto_upd = widgets.Text(
            value='',
            description='Nome Departamento:',
            disabled=False
        )

        new_nome_dpto_upd = widgets.Text(
            value='',
            description='Novo Nome:',
            disabled=False
        )

        confirm_upd_dpto = widgets.Button(description="Confirmar Atualização")

        def on_button_confirm_upd_dpto(b):
            try:
                cursor = con.cursor()
                cursor.execute("UPDATE Departamento SET nome = %s WHERE nome = %s", (new_nome_dpto_upd.value, nome_dpto_upd.value))
                con.commit()

                with output_dpto:
                    output_dpto.clear_output()
                    print("Departamento atualizado com sucesso!")

            except Exception as e:
                con.rollback()

                with output_dpto:
                    output_dpto.clear_output()
                    print("Não foi possível atualizar o departamento. Verifique o nome e tente novamente.")
                    print("Erro:", str(e))
                    traceback.print_exc()

            finally:
                cursor.close()

        # Associar ação com o botão de confirmar atualização
        confirm_upd_dpto.on_click(on_button_confirm_upd_dpto)

        # Exibir widgets para inserir nome do departamento, novo nome e botão de confirmação
        with output_dpto:
            output_dpto.clear_output()
            display(nome_dpto_upd, new_nome_dpto_upd, confirm_upd_dpto)

    except Exception as e:
        with output_dpto:
            output_dpto.clear_output()
            print("Erro ao atualizar departamento:", str(e))
            traceback.print_exc()

# Associar ação com o botão de deletar departamento
button_del_dpto.on_click(on_button_del_dpto)
button_ins_dpto.on_click(on_button_insert_dpto)
button_update_dpto.on_click(on_button_update_dpto)

# Output widget
output_dpto = widgets.Output()

# Exibir widgets e botões
display(nome_dpto, button_ins_dpto, button_list_dpto, button_del_dpto, button_update_dpto, output_dpto)


## FUNCIONARIO

In [7]:
# Widgets para a tabela Funcionario
nome = widgets.Text(
    value='',
    placeholder='Digite o nome',
    description='Nome:',
    disabled=False
)

cpf = widgets.Text(
    value='',
    placeholder='Digite seu CPF',
    description='CPF:',
    disabled=False
)

data_nascimento = widgets.DatePicker(
    description='Data de Nascimento:',
    disabled=False
)

estado_civil = widgets.Dropdown(
    value='Solteiro(a)',
    description='Estado Civil:',
    disabled=False,
    options=['Solteiro(a)', 'Casado(a)', 'Viúvo(a)', 'Outro']
)

def get_name_dpto(con):
    cursor = con.cursor()
    
    cursor.execute("SELECT id_dpto, nome FROM Departamento")
    department_data = cursor.fetchall()
    department_names = [row[1] for row in department_data]
    department_ids = [row[0] for row in department_data]
    
    cursor.close()
    
    return department_names, department_ids

department_names, department_ids = get_name_dpto(con)

departamento = widgets.Dropdown(
    disabled=False,
    options=list(zip(department_names, department_ids)),
    description='Departamento:'
)

# Widgets para a tabela Endereco
rua = widgets.Text(
    value='',
    description='Rua:',
    disabled=False
)

numero = widgets.IntText(
    value=None,
    description='Número:',
    disabled=False
)

cidade = widgets.Text(
    value='',
    description='Cidade:',
    disabled=False
)

uf = widgets.Text(
    value='',
    description='UF:',
    disabled=False
)

complemento = widgets.Text(
    value='',
    description='Complemento:',
    disabled=False
)

# Widgets para a tabela Remuneracao
salario_bruto = widgets.FloatText(
    value=None,
    description='Salário Bruto:',
    disabled=False
)

salario_liquido = widgets.FloatText(
    value=None,
    description='Salário Líquido:',
    disabled=False
)

button_ins = widgets.Button(description="Inserir", value='')
button_find_name = widgets.Button(description="Pesquisar por Nome", value='')
button_find_all = widgets.Button(description="Listar Funcionários", value='')
button_del = widgets.Button(description="Deletar", value='')
button_update = widgets.Button(description="Atualizar", value='')

def on_button_insert(b):
    cursor = con.cursor()
    
    try:
        if not nome.value.strip() or not data_nascimento.value or not cpf.value.strip() or not salario_bruto.value or not salario_liquido.value:
            output.clear_output()
            display(nome, data_nascimento, cpf, estado_civil, rua, numero, cidade, uf, complemento, salario_bruto, salario_liquido, button_ins, button_del, button_find_name, button_find_all)
            print("Por favor, preencha todos os campos obrigatórios.")
            return

        cursor.execute("INSERT INTO Funcionario (nome, data_nascimento, cpf, estado_civil, id_dpto) VALUES (%s, %s, %s, %s, %s) RETURNING id_func",
                       (nome.value, data_nascimento.value, cpf.value, estado_civil.value, departamento.value))
        id_funcionario = cursor.fetchone()[0]
        con.commit()
        
        cursor.execute("INSERT INTO Endereco (rua, numero, cidade, uf, complemento, id_func) VALUES (%s, %s, %s, %s, %s, %s)",
                       (rua.value, numero.value, cidade.value, uf.value, complemento.value, id_funcionario))
        con.commit()

        # Inserir remuneração
        cursor.execute("INSERT INTO Remuneracao (id_func, salario_bruto, salario_liquido) VALUES (%s, %s, %s)",
                       (id_funcionario, salario_bruto.value, salario_liquido.value))
        con.commit()

        output.clear_output()
        display(nome, data_nascimento, cpf, estado_civil, rua, numero, cidade, uf, complemento, button_ins, button_del, button_find_name, button_find_all)
        print("Funcionário inserido com sucesso!")

    except Exception as e:
        con.rollback()
        output.clear_output()
        display(nome, data_nascimento, cpf, estado_civil, rua, numero, cidade, uf, complemento, button_ins, button_del, button_find_name, button_find_all)
        print("Não foi possível realizar a inserção. Verifique as restrições e tente novamente.")
        print("Erro:", str(e))
        traceback.print_exc()

    finally:
        cursor.close()

def on_button_delete(b):
    try:
        cursor = con.cursor()

        if cpf.value.strip() == "":
            output.clear_output()
            display(cpf, button_del)
            print("Por favor, digite o CPF do funcionário.")
            return

        # Verificar se o funcionário existe com base no CPF
        cursor.execute("SELECT id_func FROM Funcionario WHERE cpf = %s", (cpf.value,))
        employee = cursor.fetchone()

        if employee is not None:
            id_func = employee[0]

            cursor.execute("SELECT id_func FROM Endereco WHERE id_func = %s", (id_func,))
            address = cursor.fetchone()

            if address is not None:
                cursor.execute("DELETE FROM Endereco WHERE id_func = %s", (id_func,))

            cursor.execute("SELECT id_func FROM Remuneracao WHERE id_func = %s", (id_func,))
            remuneration = cursor.fetchone()

            if remuneration is not None:
                cursor.execute("DELETE FROM Remuneracao WHERE id_func = %s", (id_func,))

            cursor.execute("DELETE FROM Funcionario WHERE id_func = %s", (id_func,))
            rows_deleted = cursor.rowcount
            con.commit()

            if rows_deleted > 0:
                output.clear_output()
                display(cpf, button_del)
                print("Funcionário deletado com sucesso!")
            else:
                output.clear_output()
                display(cpf, button_del)
                print("Nenhum funcionário encontrado com o CPF informado.")
        else:
            output.clear_output()
            display(cpf, button_del)
            print("Nenhum funcionário encontrado com o CPF informado.")

    except Exception as e:
        con.rollback()
        output.clear_output()
        display(cpf, button_del)
        print("Não foi possível deletar o funcionário. Verifique as restrições e tente novamente.")
        print("Erro:", str(e))
        traceback.print_exc()

    finally:
        cursor.close()


def on_button_find_name(b):
    try:
        cursor = con.cursor()
        cursor.execute("SELECT cpf, nome, rua, numero, cidade, uf, salario_bruto, salario_liquido FROM vw_funcionario_endereco_remuneracao WHERE nome LIKE %s", ('%' + nome.value + '%',))
        rows = cursor.fetchall()

        if rows:
            column_names = [desc[0] for desc in cursor.description]
            df = pd.DataFrame(rows, columns=column_names)
            output.clear_output()
            display(nome, button_find_name)
            display(df)
        else:
            output.clear_output()
            display(nome, button_find_name)
            print("Nenhum funcionário encontrado com o nome informado.")

    except Exception as e:
        output.clear_output()
        display(nome, button_find_name)
        print("Não foi possível realizar a consulta. Verifique as restrições e tente novamente.")
        print("Erro:", str(e))
        traceback.print_exc()

    finally:
        cursor.close()

def on_button_find_all(b):
    try:
        cursor = con.cursor()
        cursor.execute("SELECT * FROM vw_funcionario_departamento_endereco")
        rows = cursor.fetchall()
        
        if rows:
            column_names = [desc[0] for desc in cursor.description]
            df = pd.DataFrame(rows, columns=column_names)
            output.clear_output()
            display(nome, cpf, data_nascimento, estado_civil, departamento, rua, numero, cidade, uf, complemento, salario_bruto, salario_liquido, button_ins, button_del, button_find_name, button_find_all, df)
        else:
            output.clear_output()
            display(nome, cpf, data_nascimento, estado_civil, departamento, rua, numero, cidade, uf, complemento, salario_bruto, salario_liquido, button_ins, button_del, button_find_name, button_find_all)
            print("Nenhum funcionário encontrado.")
    
    except Exception as e:
        output.clear_output()        
        display(nome, cpf, data_nascimento, estado_civil, departamento, rua, numero, cidade, uf, complemento, salario_bruto, salario_liquido, button_ins, button_del, button_find_name, button_find_all)
        print("Não foi possível realizar a consulta. Verifique as restrições e tente novamente.")
        print("Erro:", str(e))
        traceback.print_exc()
    
    finally:
        cursor.close()
        
button_show_view = widgets.Button(description="Exibir Visualização")
 
def on_button_show_view(b):
    try:
        cursor = con.cursor()
        cursor.execute("SELECT * FROM funcionario")
        rows = cursor.fetchall()

        if rows:
            column_names = [desc[0] for desc in cursor.description]
            df = pd.DataFrame(rows, columns=column_names)
            output.clear_output()
            display(df)
        else:
            output.clear_output()
            print("Nenhum dado encontrado na visualização.")

    except Exception as e:
        output.clear_output()
        print("Não foi possível exibir a visualização. Verifique as restrições e tente novamente.")
        print("Erro:", str(e))
        traceback.print_exc()

    finally:
        cursor.close()

def on_button_update(b):
    cursor = con.cursor()
    
    try:
        if not id_func.value or not nome.value.strip() or not data_nascimento.value or not cpf.value.strip() or not salario_bruto.value or not salario_liquido.value:
            output.clear_output()
            display(nome, data_nascimento, cpf, estado_civil, rua, numero, cidade, uf, complemento, salario_bruto, salario_liquido, button_update, button_del, button_find_name, button_find_all)
            print("Por favor, preencha todos os campos obrigatórios.")
            return

        cursor.execute("UPDATE Funcionario SET nome = %s, data_nascimento = %s, cpf = %s, estado_civil = %s, id_dpto = %s WHERE id_func = %s",
                       (nome.value, data_nascimento.value, cpf.value, estado_civil.value, departamento.value, id_func.value))
        con.commit()
        
        cursor.execute("UPDATE Endereco SET rua = %s, numero = %s, cidade = %s, uf = %s, complemento = %s WHERE id_func = %s",
                       (rua.value, numero.value, cidade.value, uf.value, complemento.value, id_func.value))
        con.commit()

        cursor.execute("UPDATE Remuneracao SET salario_bruto = %s, salario_liquido = %s WHERE id_func = %s",
                       (salario_bruto.value, salario_liquido.value, id_func.value))
        con.commit()

        output.clear_output()
        display(nome, data_nascimento, cpf, estado_civil, rua, numero, cidade, uf, complemento, button_update, button_del, button_find_name, button_find_all)
        print("Funcionário atualizado com sucesso!")

    except Exception as e:
        con.rollback()
        output.clear_output()
        display(id_func, nome, data_nascimento, cpf, estado_civil, rua, numero, cidade, uf, complemento, button_update, button_del, button_find_name, button_find_all)
        print("Não foi possível realizar a atualização. Verifique as restrições e tente novamente.")
        print("Erro:", str(e))
        traceback.print_exc()

    finally:
        cursor.close()

button_show_view.on_click(on_button_show_view)

button_ins.on_click(on_button_insert)

button_del.on_click(on_button_delete)

button_find_name.on_click(on_button_find_name)

button_find_all.on_click(on_button_find_all)
button_update.on_click(on_button_update)

output = widgets.Output()

with output:
    output.clear_output()
    display(nome, data_nascimento, cpf, estado_civil, rua, numero, cidade, uf, complemento, salario_bruto, salario_liquido, button_ins, button_update, button_del, button_find_name, button_find_all)

display(output)


Output()

## TAREFA

In [36]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output
import psycopg2

# Função para obter os nomes dos funcionários
def get_name_funcionario():
    try:
        cursor = conn.cursor()
        cursor.execute("SELECT id_func, nome FROM Funcionario")
        employees = cursor.fetchall()

        employee_names = [row[1] for row in employees]
        employee_ids = [row[0] for row in employees]

        employee_dropdown = widgets.Dropdown(
            options=list(zip(employee_ids, employee_names)),
            value=None,
            description='Funcionário:',
            disabled=False
        )

        cursor.close()

        return employee_dropdown

    except Exception as e:
        print("Erro ao obter nomes de funcionários:", str(e))

# Função para criar uma nova tarefa
def create_tarefa(descricao, data, status, id_func):
    try:
        cursor = conn.cursor()
        cursor.execute("""
            INSERT INTO Tarefas (descricao, data, status, id_func)
            VALUES (%s, %s, %s, %s)
        """, (descricao, data, status, id_func))
        conn.commit()
        print("Tarefa criada com sucesso!")
    except Exception as error:
        print("Erro ao criar tarefa:", error)

# Função para listar todas as tarefas
def read_tarefas(button):
    try:
        cursor = conn.cursor()
        cursor.execute("SELECT * FROM Tarefas")
        rows = cursor.fetchall()
        if rows:
            column_names = [desc[0] for desc in cursor.description]
            df = pd.DataFrame(rows, columns=column_names)
            display(df)
        else:
            print("Nenhuma tarefa encontrada.")
    except Exception as error:
        print("Erro ao listar tarefas:", error)

# Função para atualizar uma tarefa existente
def update_tarefa(id_tarefa, descricao, data, status, id_func):
    try:
        cursor = conn.cursor()
        cursor.execute("""
            UPDATE Tarefas
            SET descricao = %s, data = %s, status = %s, id_func = %s
            WHERE id_tarefa = %s
        """, (descricao, data, status, id_func, id_tarefa))
        conn.commit()
        print("Tarefa atualizada com sucesso!")
    except Exception as error:
        print("Erro ao atualizar tarefa:", error)

# Função para excluir uma tarefa
def delete_tarefa(id_tarefa):
    try:
        cursor = conn.cursor()
        cursor.execute("""
            DELETE FROM Tarefas
            WHERE id_tarefa = %s
        """, (id_tarefa,))
        conn.commit()
        print("Tarefa excluída com sucesso!")
    except Exception as error:
        print("Erro ao excluir tarefa:", error)

# Função para criar uma nova tarefa usando uma interface interativa
def create_tarefa_interactive(button):
    descricao_input = widgets.Text(description='Descrição:')
    data_input = widgets.DatePicker(description='Data:')
    status_input = widgets.Dropdown(options=['Em andamento', 'Concluída', 'Não iniciada'], description='Status:')
    employee_dropdown = get_name_funcionario()
    button_create = widgets.Button(description='Criar')

    def on_button_create_clicked(b):
        create_tarefa(descricao_input.value, data_input.value, status_input.value, employee_dropdown.value)
        clear_output()
        display(create_tarefa_button, update_tarefa_button, delete_tarefa_button, button_listar_tarefas)

    button_create.on_click(on_button_create_clicked)

    display(descricao_input, data_input, status_input, employee_dropdown, button_create)

# Função para atualizar uma tarefa existente usando uma interface interativa
def update_tarefa_interactive(button):
    id_tarefa_input = widgets.IntText(description='ID:')
    descricao_input = widgets.Text(description='Descrição:')
    data_input = widgets.DatePicker(description='Data:')
    status_input = widgets.Dropdown(options=['Em andamento', 'Concluída', 'Não iniciada'], description='Status:')
    employee_dropdown = get_name_funcionario()
    button_update = widgets.Button(description='Atualizar')

    def on_button_update_clicked(b):
        update_tarefa(id_tarefa_input.value, descricao_input.value, data_input.value, status_input.value, employee_dropdown.value)
        clear_output()
        display(create_tarefa_button, update_tarefa_button, delete_tarefa_button, button_listar_tarefas)

    button_update.on_click(on_button_update_clicked)

    display(id_tarefa_input, descricao_input, data_input, status_input, employee_dropdown, button_update)

# Função para excluir uma tarefa usando uma interface interativa
def delete_tarefa_interactive(button):
    id_tarefa_input = widgets.IntText(description='ID:')
    button_delete = widgets.Button(description='Excluir')

    def on_button_delete_clicked(b):
        delete_tarefa(id_tarefa_input.value)
        clear_output()
        display(create_tarefa_button, update_tarefa_button, delete_tarefa_button, button_listar_tarefas)

    button_delete.on_click(on_button_delete_clicked)

    display(id_tarefa_input, button_delete)

# Estabelecer a conexão com o banco de dados
host = 'localhost'
dbname = 'postgres'
user = 'postgres'
password = '2505'

conn = psycopg2.connect(
    host=host,
    dbname=dbname,
    user=user,
    password=password
)

# Botão para listar todas as tarefas
button_listar_tarefas = widgets.Button(description='Listar Tarefas')

button_listar_tarefas.on_click(read_tarefas)

# Criar uma nova tarefa usando interface interativa
create_tarefa_button = widgets.Button(description='Criar Tarefa')
create_tarefa_button.on_click(create_tarefa_interactive)

# Atualizar uma tarefa existente usando interface interativa
update_tarefa_button = widgets.Button(description='Atualizar Tarefa')
update_tarefa_button.on_click(update_tarefa_interactive)

# Excluir uma tarefa usando interface interativa
delete_tarefa_button = widgets.Button(description='Excluir Tarefa')
delete_tarefa_button.on_click(delete_tarefa_interactive)

display(create_tarefa_button, update_tarefa_button, delete_tarefa_button, button_listar_tarefas)


Button(description='Criar Tarefa', style=ButtonStyle())

Button(description='Atualizar Tarefa', style=ButtonStyle())

Button(description='Excluir Tarefa', style=ButtonStyle())

Button(description='Listar Tarefas', style=ButtonStyle())

Erro ao listar tarefas: current transaction is aborted, commands ignored until end of transaction block

Erro ao listar tarefas: current transaction is aborted, commands ignored until end of transaction block

Erro ao listar tarefas: current transaction is aborted, commands ignored until end of transaction block



In [24]:
def insert_beneficio(b):
    tipo = tipo_benef.value
    nome = nome_benef.value
    valor = valor_benef.value
    id_func = id_func_benef.value
    try:
        con = connect()     
        if con:
            cursor = con.cursor()
            if id_func:
                cursor.execute(
                    "INSERT INTO Beneficios (tipo, nome, valor_beneficio, id_func) VALUES (%s, %s, %s, %s)",
                    (tipo, nome, valor, id_func)
                )
            else:
                cursor.execute(
                    "INSERT INTO Beneficios (tipo, nome, valor_beneficio) VALUES (%s, %s, %s)",
                    (tipo, nome, valor)
                )
            con.commit()
            print("Benefício inserido com sucesso.")

    except Exception as e:
        print("Erro ao inserir benefício. Verifique os valores e tente novamente.")
        print("Erro:", str(e))
        traceback.print_exc()

    finally:
        cursor.close()
        con.close()

def delete_beneficio(b):
    id_benef = id_beneficio_del.value
    try:
        con = connect()
        if con:
            cursor = con.cursor()
            cursor.execute("DELETE FROM Beneficios WHERE id_benef = %s", (id_benef,))
            con.commit()
            print("Benefício deletado com sucesso.")

    except Exception as e:
        print("Erro ao deletar benefício. Verifique o ID e tente novamente.")
        print("Erro:", str(e))
        traceback.print_exc()

    finally:
        cursor.close()
        con.close()

# Function to list all benefits
def list_beneficios(b):
    try:
        con = connect()
        if con:
            cursor = con.cursor()
            cursor.execute("SELECT * FROM Beneficios")
            rows = cursor.fetchall()

            if rows:
                column_names = [desc[0] for desc in cursor.description]
                df = pd.DataFrame(rows, columns=column_names)
                display(df)
            else:
                print("Nenhum benefício encontrado.")

    except Exception as e:
        print("Erro ao listar benefícios.")
        print("Erro:", str(e))
        traceback.print_exc()

    finally:
        cursor.close()
        con.close()

def update_beneficio(b):
    id_benef = id_beneficio_update.value
    tipo = tipo_benef.value
    nome = nome_benef.value
    valor = valor_benef.value
    id_func = id_func_benef.value
    
    try:
        con = connect()
        
        if con:
            cursor = con.cursor()
            if id_func:
                cursor.execute(
                    "UPDATE Beneficios SET tipo = %s, nome = %s, valor_beneficio = %s, id_func = %s WHERE id_benef = %s",
                    (tipo, nome, valor, id_func, id_benef)
                )
            else:
                cursor.execute(
                    "UPDATE Beneficios SET tipo = %s, nome = %s, valor_beneficio = %s WHERE id_benef = %s",
                    (tipo, nome, valor, id_benef)
                )
            con.commit()

            if cursor.rowcount > 0:
                print("Benefício atualizado com sucesso.")
            else:
                print("Nenhum benefício encontrado com o ID informado.")

    except (Exception, psycopg2.Error) as e:
        print("Erro ao atualizar benefício. Verifique os valores e tente novamente.")
        print("Erro:", str(e))
        traceback.print_exc()

    finally:
        cursor.close()
        con.close()

output = widgets.Output()

tipo_benef = widgets.Text(value='', description='Tipo:', disabled=False)
nome_benef = widgets.Text(value='', description='Nome:', disabled=False)
valor_benef = widgets.FloatText(value=0, description='Valor:', disabled=False)
id_func_benef = widgets.IntText(value=0, description='ID Funcionário:', disabled=False)
id_beneficio_del = widgets.IntText(value=0, description='ID Benefício:', disabled=False)
id_beneficio_update = widgets.IntText(value=0, description='ID Benefício:', disabled=False)

button_list_beneficios = widgets.Button(description="Listar Benefícios")
button_ins_beneficio = widgets.Button(description="Inserir Benefício")
button_del_beneficio = widgets.Button(description="Deletar Benefício")
button_update_beneficio = widgets.Button(description="Atualizar Benefício")

button_list_beneficios.on_click(list_beneficios)
button_ins_beneficio.on_click(insert_beneficio)
button_del_beneficio.on_click(delete_beneficio)
button_update_beneficio.on_click(update_beneficio)

display(tipo_benef, nome_benef, valor_benef, id_func_benef, id_beneficio_del, output)
display(button_ins_beneficio, button_list_beneficios, button_del_beneficio, button_update_beneficio)


Text(value='', description='Tipo:')

Text(value='', description='Nome:')

FloatText(value=0.0, description='Valor:')

IntText(value=0, description='ID Funcionário:')

IntText(value=0, description='ID Benefício:')

Output()

Button(description='Inserir Benefício', style=ButtonStyle())

Button(description='Listar Benefícios', style=ButtonStyle())

Button(description='Deletar Benefício', style=ButtonStyle())

Button(description='Atualizar Benefício', style=ButtonStyle())